<a href="https://colab.research.google.com/github/DanulR/Danul-Rochana-AI-Projects/blob/main/SentimentAnalysisThreadsApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re


In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets/37000_reviews_of_thread_app.csv')

In [ ]:
dataset.head()

,Unnamed: 0,source,review_id,user_name,review_title,review_description,rating,thumbs_up,review_date,developer_response,developer_response_date,appVersion,laguage_code,country_code
0,0,Google Play,7cd90e5b-4829-43b9-9fb4-c8c6d1e339c1,Eddie Clark Jr.,NaN,Good,5,0.0,2023-08-07 19:14:36,NaN,NaN,294.0.0.27.110,en,us
1,1,Google Play,6deb8265-2bac-4524-bcb6-f90829fa4e69,Rasa RT,NaN,Weak copy of Twitter,1,0.0,2023-08-07 19:07:04,NaN,NaN,NaN,en,us
2,2,Google Play,91ef61ce-0f05-4f3b-b3d3-5d19cd408ab8,SITI NUR HAFIZA BINTI AZIZ,NaN,i wish threads have a save button for images a...,3,0.0,2023-08-07 18:57:07,NaN,NaN,294.0.0.27.110,en,us
3,3,Google Play,b7721b78-6b77-4f8c-a1d3-a854af4c1f0f,Asap Khalifah,NaN,Love it,5,0.0,2023-08-07 18:37:16,NaN,NaN,NaN,en,us
4,4,Google Play,c89ef522-c94c-4171-878f-1d672dce7f11,Syed Hussein,NaN,Very god,5,0.0,2023-08-07 18:14:15,NaN,NaN,NaN,en,us


In [ ]:
dataset = dataset[['review_description','rating']]

In [ ]:
dataset['review_description'] = dataset['review_description'].apply(lambda x: x.lower())
dataset['review_description'] = dataset['review_description'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [ ]:
for idx,row in dataset.iterrows():
    row[0] = row[0].replace('rt',' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(dataset['review_description'].values)
X = tokenizer.texts_to_sequences(dataset['review_description'].values)
X = pad_sequences(X)

In [ ]:
embed_dim = 128
lstm_out = 196


model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(5,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 372, 128)          256000    
                                                                 
 spatial_dropout1d (SpatialD  (None, 372, 128)         0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 5)                 985       
                                                                 
Total params: 511,785
Trainable params: 511,785
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
Y = pd.get_dummies(dataset['rating']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(24751, 372) (24751, 5)
(12192, 372) (12192, 5)


In [ ]:
batch_size = 1000

model.fit(X_train, Y_train, epochs = 125, batch_size=batch_size, verbose = 2)

Epoch 1/125
25/25 - 57s - loss: 1.3299 - accuracy: 0.4799 - 57s/epoch - 2s/step
Epoch 2/125
25/25 - 53s - loss: 1.1680 - accuracy: 0.5528 - 53s/epoch - 2s/step
Epoch 3/125
25/25 - 48s - loss: 1.0486 - accuracy: 0.6191 - 48s/epoch - 2s/step
Epoch 4/125
25/25 - 46s - loss: 0.9760 - accuracy: 0.6588 - 46s/epoch - 2s/step
Epoch 5/125
25/25 - 47s - loss: 0.9026 - accuracy: 0.6854 - 47s/epoch - 2s/step
Epoch 6/125
25/25 - 46s - loss: 0.8686 - accuracy: 0.6977 - 46s/epoch - 2s/step
Epoch 7/125
25/25 - 49s - loss: 0.8521 - accuracy: 0.7040 - 49s/epoch - 2s/step
Epoch 8/125
25/25 - 48s - loss: 0.8353 - accuracy: 0.7105 - 48s/epoch - 2s/step
Epoch 9/125
25/25 - 48s - loss: 0.8281 - accuracy: 0.7118 - 48s/epoch - 2s/step
Epoch 10/125
25/25 - 54s - loss: 0.8207 - accuracy: 0.7161 - 54s/epoch - 2s/step
Epoch 11/125
25/25 - 45s - loss: 0.8141 - accuracy: 0.7173 - 45s/epoch - 2s/step
Epoch 12/125
25/25 - 45s - loss: 0.8077 - accuracy: 0.7191 - 45s/epoch - 2s/step
Epoch 13/125
25/25 - 45s - loss: 0.80

In [ ]:


validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))



11/11 - 2s - loss: 1.7512 - accuracy: 0.6486 - 2s/epoch - 177ms/step
score: 1.75
acc: 0.65


In [ ]:
def predict_sentiment(review_text, model, tokenizer):
    review_text = review_text.lower()
    review_text = re.sub('[^a-zA-Z0-9\s]', '', review_text)
    review_text = review_text.replace('rt', ' ')

    review_seq = tokenizer.texts_to_sequences([review_text])
    review_padded = pad_sequences(review_seq, maxlen=X.shape[1])

    sentiment_probs = model.predict(review_padded)[0]
    predicted_sentiment = np.argmax(sentiment_probs) + 1

    return predicted_sentiment

# Example usage:
sample_review = "i wish threads have a save button for images and videos. i would like to save the memes. plssss consider this proposition. thank you."
predicted_rating = predict_sentiment(sample_review, model, tokenizer)
print("Predicted Rating:", predicted_rating)

1/1 [==============================] - 0s 353ms/step
Predicted Rating: 3
